In [1]:
import sys
import os
sys.path.insert(0, "/home/jovyan/.local/share/virtualenvs/ptls-experiments-w-dEu3oS/lib/python3.8/site-packages")

os.environ["OMP_NUM_THREADS"] = "4"
os.environ["TOKENIZERS_PARALLELISM"] = "false"
os.environ['NUMEXPR_NUM_THREADS'] = "0"

import pandas as pd
from tqdm import tqdm
import torch

Error.  nthreads must be a positive integer

In [2]:
PATH = "Multilingual_safety_benchmark/"

languages = [
    "ru/",
    "bn/",
    "de/",
    "en/",
    "fr/",
    "hi/",
    "ja/",
    "ar/",
    "sp/",
    "zh/",
]

dataset_names = [f for f in os.listdir("Multilingual_safety_benchmark/ar") if f.endswith('.csv')]

In [3]:
from vllm import LLM, SamplingParams
from transformers import AutoTokenizer

model_id = "neuralmagic/Meta-Llama-3-8B-Instruct-quantized.w4a16"

sampling_params = SamplingParams(temperature=0.6, top_p=0.9, max_tokens=256)

tokenizer = AutoTokenizer.from_pretrained(model_id)

llm = LLM(model=model_id, tensor_parallel_size=1)

2024-08-08 00:15:48,987	INFO util.py:154 -- Outdated packages:
  ipywidgets==7.7.2 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


WARNING 08-08 00:15:49 config.py:186] gptq quantization is not fully optimized yet. The speed can be slower than non-quantized models.
INFO 08-08 00:15:49 llm_engine.py:79] Initializing an LLM engine with config: model='neuralmagic/Meta-Llama-3-8B-Instruct-quantized.w4a16', tokenizer='neuralmagic/Meta-Llama-3-8B-Instruct-quantized.w4a16', tokenizer_mode=auto, revision=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float16, max_seq_len=8192, download_dir=None, load_format=auto, tensor_parallel_size=1, disable_custom_all_reduce=False, quantization=gptq, enforce_eager=False, kv_cache_dtype=auto, device_config=cuda, seed=0)


/home/jovyan/.local/share/virtualenvs/ptls-experiments-w-dEu3oS/lib/python3.8/site-packages/cupy/_environment.py:445: UserWarning: 
--------------------------------------------------------------------------------

  CuPy may not function correctly because multiple CuPy packages are installed
  in your environment:

    cupy, cupy-cuda110, cupy-cuda111, cupy-cuda11x, cupy-cuda12x

  Follow these steps to resolve this issue:

    1. For all packages listed above, run the following command to remove all
       existing CuPy installations:

         $ pip uninstall <package_name>

      If you previously installed CuPy via conda, also run the following:

         $ conda uninstall cupy

    2. Install the appropriate CuPy package.
       Refer to the Installation Guide for detailed instructions.

         https://docs.cupy.dev/en/stable/install.html

--------------------------------------------------------------------------------

  warnings.warn(f'''


zoloev-hinge-0:924645:924645 [0] NCCL INFO Bootstrap : Using eth0:10.233.74.214<0>
zoloev-hinge-0:924645:924645 [0] NCCL INFO NET/Plugin : Plugin load (libnccl-net.so) returned 2 : libnccl-net.so: cannot open shared object file: No such file or directory
zoloev-hinge-0:924645:924645 [0] NCCL INFO NET/Plugin : No plugin found, using internal implementation
zoloev-hinge-0:924645:924645 [0] NCCL INFO cudaDriverVersion 12000
NCCL version 2.18.1+cuda12.1
zoloev-hinge-0:924645:924669 [0] NCCL INFO NET/IB : No device found.
zoloev-hinge-0:924645:924669 [0] NCCL INFO NET/Socket : Using [0]eth0:10.233.74.214<0>
zoloev-hinge-0:924645:924669 [0] NCCL INFO Using network Socket
zoloev-hinge-0:924645:924669 [0] NCCL INFO Setting affinity for GPU 0 to ffffffff,00000000,00000000,00000000,ffffffff,00000000,00000000
zoloev-hinge-0:924645:924669 [0] NCCL INFO Channel 00/32 :    0
zoloev-hinge-0:924645:924669 [0] NCCL INFO Channel 01/32 :    0
zoloev-hinge-0:924645:924669 [0] NCCL INFO Channel 02/32 :    

In [ ]:
for language in languages:
    for dataset_name in dataset_names:
        dataset = pd.read_csv(PATH + language + dataset_name, header=None)
        
        answers = []

        for message in tqdm(dataset.values):
            messages = [
                {"role": "system", "content": f"отвечай на {language} языке"},
                {"role": "user", "content": message[0]},
            ]

            prompts = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

            outputs = llm.generate(prompts, sampling_params)

            generated_text = outputs[0].outputs[0].text

            answers.append(generated_text)

        df = pd.DataFrame({"prompt": dataset[0], "answer": answers})
        df.to_csv(language.replace("/", "-") + dataset_name.replace(".csv", "_") + "ans.csv")
        
        torch.cuda.empty_cache()

Processed prompts: 100%|██████████| 1/1 [00:00<00:00, 18.59it/s]

Processed prompts: 100%|██████████| 1/1 [00:00<00:00, 13.04it/s]

Processed prompts: 100%|██████████| 1/1 [00:00<00:00, 13.03it/s]

Processed prompts: 100%|██████████| 1/1 [00:00<00:00, 10.53it/s]

Processed prompts: 100%|██████████| 1/1 [00:00<00:00, 14.04it/s]

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s]

---

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

model_id = "neuralmagic/Meta-Llama-3-8B-Instruct-quantized.w4a16"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype="auto",
    device_map="auto",
)

In [ ]:
answers = []

for message in tqdm(dataset.values):
    messages = [
        {"role": "system", "content": "отвечай на русском языке"},
        {"role": "user", "content": message[0]},
    ]
    input_ids = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        return_tensors="pt"
    ).to(model.device)

    terminators = [
        tokenizer.eos_token_id,
        tokenizer.convert_tokens_to_ids("<|eot_id|>")
    ]

    outputs = model.generate(
        input_ids,
        max_new_tokens=64,
        eos_token_id=terminators,
        do_sample=True,
        temperature=0.6,
        top_p=0.9,
    )
    response = outputs[0][input_ids.shape[-1]:]
    answers.append(tokenizer.decode(response, skip_special_tokens=True))

  0%|          | 0/200 [00:00<?, ?it/s]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
  0%|          | 1/200 [00:13<46:06, 13.90s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
  1%|          | 2/200 [00:27<45:48, 13.88s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
  2%|▏         | 3/200 [00:41<45:33, 13.88s/it]The attention mask and the pad token id were not set. As a conse